cnn_deep.ipynbから畳み込み層とプーリング層をさらに増やす

In [ ]:
#Cell_1.
# -*- coding: utf-8 -*-
import os
import cv2
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix

import tensorflow
seedval = 13
tensorflow.random.set_seed(seedval)
np.random.seed(seedval)
random.seed(seedval)
os.environ["PYTHONHASHSEED"] = str(seedval)

In [ ]:
#Cell_2.
IMAGESIZE = 256  #読み込んだ画像は(IMAGESIZE,IMAGESIZE)のサイズにリサイズする

# ディレクトリ内の画像を読み込む
# inputpath: ディレクトリ文字列, imagesize: 画像サイズ, type_color: ColorかGray
def load_images(inputpath, imagesize, type_color):
    imglist = []
    filenamelist = []

    for root, dirs, files in os.walk(inputpath):
        for fn in sorted(files):
            bn, ext = os.path.splitext(fn)
            if ext not in [".bmp", ".BMP", ".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"]:
                continue

            filename = os.path.join(root, fn)

            if type_color == 'Color':
                # カラー画像の場合
                testimage = cv2.imread(filename, cv2.IMREAD_COLOR)
                # サイズ変更
                height, width = testimage.shape[:2]
                testimage = cv2.resize(testimage, (imagesize, imagesize), interpolation = cv2.INTER_AREA)  #主に縮小するのでINTER_AREA使用
                testimage = np.asarray(testimage, dtype=np.float64)
                # 色チャンネル，高さ，幅に入れ替え．data_format="channels_first"を使うとき必要
                #testimage = testimage.transpose(2, 0, 1)
                # チャンネルをbgrの順からrgbの順に変更
                testimage = testimage[:,:,::-1]

            elif type_color == 'Gray':
                # グレースケール画像の場合
                testimage = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                # サイズ変更
                height, width = testimage.shape[:2]
                testimage = cv2.resize(testimage, (imagesize, imagesize), interpolation = cv2.INTER_AREA)  #主に縮小するのでINTER_AREA使用
                # チャンネルの次元がないので1次元追加する
                testimage = np.asarray([testimage], dtype=np.float64)
                testimage = np.asarray(testimage, dtype=np.float64).reshape((imagesize, imagesize, 1))
                # チャンネル，高さ，幅に入れ替え．data_format="channels_first"を使うとき必要
                #testimage = testimage.transpose(2, 0, 1)

            imglist.append(testimage)
            filenamelist.append(fn)
    imgsdata = np.asarray(imglist, dtype=np.float32)

    return imgsdata, filenamelist  # 画像リストとファイル名のリストを返す


In [ ]:
#Cell_3.
#%%
### データ準備 ###
print('*** Loading images ***')

# 画像読み込みとラベル値作成
# クラス0
image0, filenames_image0 = load_images('./sundatabase_negative/', IMAGESIZE, 'Color')
label0 = np.full(image0.shape[0], 0)    #画像数と同数のラベル値を含むリスト

# クラス1
image1, filenames_image1 = load_images('./sundatabase_positive/', IMAGESIZE, 'Color')
label1 = np.full(image1.shape[0], 1)    #画像数と同数のラベル値を含むリスト


# 画像，ラベル値，ファイル名それぞれ1つの配列にまとめる
image_all = np.concatenate([image0, image1], axis=0)
label_all = np.append(label0, label1)
filename_all = filenames_image0 + filenames_image1


In [ ]:
#Cell_4.
indices = np.array(range(image_all.shape[0]))
image_train, image_test, label_train, label_test, index_train, index_test = train_test_split(image_all, label_all, indices, test_size=0.2)

In [ ]:
#Cell_5.
print(indices)
print(len(label_train))
print(len(label_test))

In [ ]:
#Cell_6.
# testデータのファイル名リスト作成
filenames_test = []
for i in range(len(index_test)):
    filenames_test.append(filename_all[index_test[i]])


# 画像の画素値を0-1に正規化
image_train /= 255.0
image_test /= 255.0

# ラベルをone hot vector形式に変換
label_train_binary = to_categorical(label_train)
label_test_binary = to_categorical(label_test)

print('Loaded images: ' + repr(image_train.shape[0]) + ' for training and ' + repr(image_test.shape[0]) + ' for testing.')


In [ ]:
#Cell_7.
### 画像分類モデル定義と学習処理の実行 ###
print('*** Training ***')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 層の数を増やした分類CNN（Sequential形式）
model = Sequential()

# 1st Convolutional block
model.add(Conv2D(32, (3, 3), strides=1, padding='same',
                 activation='relu', input_shape=(IMAGESIZE, IMAGESIZE, 3)))
model.add(MaxPooling2D(pool_size=(4, 4), strides=4))

# 2nd Convolutional block
model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides=4))

# 3rd Convolutional block
model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides=4))

# 4th Convolutional block
model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=4))

# Flatten → Fully connected layers
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(2, activation='softmax'))

# モデル構造を表示
print(model.summary())


In [ ]:
%%time
#Cell_8.
n_epochs=100
val_split=0.2
batch_size=128
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping

history_file="history-w{}.csv".format(IMAGESIZE)
model.compile(loss='binary_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])
cl=CSVLogger(history_file)
es=EarlyStopping(monitor='val_loss', patience=12, verbose=1)

fit_log=model.fit(image_train, label_train_binary, batch_size=batch_size,
                  epochs=n_epochs, validation_split=val_split,
                  callbacks=[cl, es])
#model14.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# training実行
#training = model.fit(image_train, label_train_binary, epochs=100, batch_size=2, verbose=1)

In [ ]:
#Cell_9.
loss=fit_log.history['loss']
val_loss=fit_log.history['val_loss']
acc=fit_log.history['accuracy']
val_acc=fit_log.history['val_accuracy']
epoch=range(len(loss))

plt.plot(epoch, loss, label='train')
plt.plot(epoch, val_loss, label='valid')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epoch, acc, label='train')
plt.plot(epoch, val_acc, label='valid')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#Cell_10.
score=model.evaluate(image_test, label_test_binary, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
#Cell_11.
y_test_pred=model.predict(image_test, verbose=0)
ct = pd.crosstab(label_test_binary.argmax(axis=1),
                 y_test_pred.argmax(axis=1))
display(ct)

In [ ]:
#Cell_12.
# testデータの分類結果をcsvファイルに書き出し
f = open('./result_deep.csv', 'w')
writer = csv.writer(f, lineterminator='\n')

savedata = ['filename', 'true_class', 'estimate_class', 'estimate_likelihood_max', 'estimate_likelihood_class0', 'estimate_likelihood_class1']
writer.writerow(savedata)

for i in range(len(image_test)):
    savedata = [filenames_test[i], label_test[i], np.argmax(y_test_pred[i]), np.max(y_test_pred[i]), y_test_pred[i][0], y_test_pred[i][1]]
    writer.writerow(savedata)

f.close()